In [1]:
import utils

f0_predictor = utils.get_f0_predictor('rmvpe',
                                        sampling_rate=44100,
                                        hop_length=512,
                                        device='cuda',
                                        threshold=0.05)

In [2]:
import librosa
wav, sr = librosa.load("notebook/test_audios/ref_audio/24k_xigua.wav", sr=44100)

In [3]:
import time


f0, uv = f0_predictor.compute_f0_uv(wav)


In [8]:
import yaml
config = yaml.safe_load(open('configs/pretrained/config.yaml'))

In [11]:
config.update()

In [2]:
all_data = []
with open("Data/backup/raw_text_en.list", 'r') as f:
    for line in f:
        all_data.append(line.strip())
with open("Data/raw_text.list", 'r') as f:
    for line in f:
        all_data.append(line.strip())

In [4]:
import random

random.shuffle(all_data)

valid = all_data[:10]
train = all_data[10:]

In [6]:
with open("Data/sovits_svc/train.list",'w') as f:
    for line in train:
        line = line.strip()
        f.write(line + '\n')

with open("Data/sovits_svc/dev.list",'w') as f:
    for line in valid:
        line = line.strip()
        f.write(line + '\n')

In [3]:
with open("Data/raw_text.list",'w') as f:
    for line in all_data:
        line = line.strip()
        f.write(line + '\n')

In [13]:
import os
from pathlib import Path


dir = "ckpts/sunyanzi/audio_slice/sunyanzi"
p = Path(dir)

In [9]:
def move_files_simple(source_dir, target_dir):
    source_path = Path(source_dir)
    target_path = Path(target_dir)
    
    # 确保目标目录存在
    target_path.mkdir(parents=True, exist_ok=True)
    
    # 移动所有文件
    for file_path in source_path.glob("*.wav"):
        if file_path.is_file():
            file_path.rename(target_path / file_path.name)
            print(f"Moved: {file_path.name}")

In [ ]:
move_files_simple(p, p.parent.parent/'sunyanzi')

In [5]:
from pathlib import Path

t = Path("ckpts/sunyanzi/audio_slice/sunyanzi/sunyanzi_000000.wav")

In [6]:
"_".join(t.stem.split('_')[:2]) + '_f0.npy'

'sunyanzi_000000_f0.npy'

In [9]:
any(t.glob("g_*.pth"))

False

In [8]:
 list(t.glob("g_*.pth"))

[]

In [ ]:
import sys
import os
import torch
import torch.backends.cudnn as cudnn

def check_environment():
    print("=== Python Environment ===")
    print(f"Python executable: {sys.executable}")
    print(f"Python version: {sys.version}")
    
    print("\n=== PyTorch Information ===")
    print(f"PyTorch location: {torch.__file__}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"CUDNN version: {cudnn.version()}")
    
    print("\n=== Environment Variables ===")
    print(f"CONDA_PREFIX: {os.environ.get('CONDA_PREFIX', 'Not set')}")
    print(f"LD_LIBRARY_PATH: {os.environ.get('LD_LIBRARY_PATH', 'Not set')}")
    
    if torch.cuda.is_available():
        print("\n=== CUDA Device Information ===")
        print(f"Current device: {torch.cuda.current_device()}")
        print(f"Device name: {torch.cuda.get_device_name()}")
        try:
            print(f"Device capability: {torch.cuda.get_device_capability()}")
        except AttributeError:
            pass
    
    print("\n=== Python Path ===")
    for path in sys.path:
        print(path)
    
    # 尝试定位 CUDA 相关库
    print("\n=== CUDA Library Check ===")
    torch_path = os.path.dirname(torch.__file__)
    print(f"Looking for CUDA libraries in: {torch_path}")
    
    # 检查常见的库文件位置
    cuda_paths = [
        os.path.join(torch_path, 'lib'),
        os.environ.get('CONDA_PREFIX', ''),
        '/usr/local/cuda/lib64',
        '/usr/lib/x86_64-linux-gnu'
    ]
    
    for path in cuda_paths:
        if os.path.exists(path):
            print(f"\nChecking {path}:")
            cuda_libs = [f for f in os.listdir(path) if 'cuda' in f.lower() or 'cudnn' in f.lower()]
            for lib in cuda_libs:
                print(f"Found: {lib}")

if __name__ == "__main__":
    check_environment()

In [1]:
import torch

In [2]:
# sovits底膜专用
def remove_optimizer(path):
    ckpt = torch.load(path, map_location='cpu')
    ckpt['optimizer'] = None
    torch.save(ckpt, path)
    

In [4]:
remove_optimizer("/datadisk/liujunjie/so-vits-svc/ckpts/test/models/D_120000.pth")

In [1]:
from pathlib import Path

model_dir = Path("ckpts/sunyanzi/diffusion")

In [5]:
f_list = list(model_dir.glob("*.pt"))

f_list.sort(
    key=lambda f: int("".join(filter(str.isdigit, f))))

In [1]:
import os

len(os.listdir("/datadisk/liujunjie/so-vits-svc/raw_audios/preprocess/sing_10s/timbre_feature"))

69729

In [ ]:
from multiprocessing.managers import BaseManager
import time

def connect_to_server():
    # 定义QueueManager
    class QueueManager(BaseManager):
        pass

    # 注册队列
    QueueManager.register('get_train_queue')
    QueueManager.register('get_dev_queue')

    # 创建管理器实例
    manager = QueueManager(
        address=('127.0.0.1', 12345),
        authkey=b'liujunjieabracadabra'
    )

    # 尝试连接
    try:
        manager.connect()
        print("成功连接到服务器")
        return manager
    except Exception as e:
        print(f"连接失败: {e}")
        return None

def main():
    # 连接服务器
    manager = connect_to_server()
    if not manager:
        return

    # 获取队列
    train_queue = manager.get_train_queue()
    # dev_queue = manager.get_dev_queue()

    # 持续从队列获取数据
    try:
        while True:
            # 尝试从训练队列获取数据
            try:
                train_data = train_queue.get(timeout=5)  # 5秒超时
                print("收到训练数据:", train_data)
                # 这里处理train_data...
            except:
                print("训练队列暂无数据")

            # # 尝试从开发队列获取数据
            # try:
            #     dev_data = dev_queue.get(timeout=5)  # 5秒超时
            #     print("收到开发数据:", dev_data)
            #     # 这里处理dev_data...
            # except:
            #     print("开发队列暂无数据")

            time.sleep(1)  # 避免过于频繁的请求

    except KeyboardInterrupt:
        print("程序终止")
    except Exception as e:
        print(f"运行出错: {e}")

if __name__ == '__main__':
    main()

In [4]:
import os

len(os.listdir("/datadisk/liujunjie/so-vits-svc/Data/sovits_svc/f0s"))

1026100

In [1]:
data = []
with open("Data/raw_text.list",'r') as f:
    for line in f:
        data.append(line.strip())
len(data)

1024580

In [8]:
import random
random.shuffle(data)

data = data[:60000]

new_data = []

for line in data:
    line = line.strip().split("|")
    if len(line[2]) < 30:
        new_data.append(line)
    if len(new_data) > 60000:
        break



In [24]:
import json

sing_spk_info = json.load(open("Data/backup/sing_spk_info.json",'r'))
print(len(sing_spk_info))



952


In [25]:
sing_data = []
with open("/datadisk/liujunjie/so-vits-svc/Data/sing_raw_text.list",'r') as f:
    for line in f:
        line = line.strip()
        sing_data.append(line)
        

In [26]:
for line in new_data:
    # line = line.strip().split("|")
    if line[2] not in sing_spk_info:
        sing_spk_info[line[2]] = len(sing_spk_info)
    sing_data.append("|".join(line))

json.dump(sing_spk_info,open("Data/sovits_svc/sing_spk_info.json",'w'),indent=4,ensure_ascii=False)


In [27]:
random.shuffle(sing_data)

with open("Data/sing_raw_text.list",'w') as f:
    for line in sing_data:
        f.write(line + '\n')

In [25]:
import os

len(os.listdir("Data/sovits_svc/ssls"))

1144036